In [8]:
import pandas as pd
INPUT_FILE = 'categories.xlsx'     
SHEET_NAME = 'Sheet1'                   
HIERARCHY_COL = 'Category (en)'         
OUTPUT_FILE = 'categories_cleaned.xlsx'

def split_hierarchies():
    print(f"Reading {INPUT_FILE}...")
    try:
        df = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Error loading file: {e}")
        return
    
    if HIERARCHY_COL not in df.columns:
        print(f"Error: Column '{HIERARCHY_COL}' not found in the Excel file.")
        print(f"Available columns are: {list(df.columns)}")
        return
    print("Splitting hierarchies...")

    # 1. Split the column by '/'
    # expand=True puts the results into separate columns automatically
    split_columns = df[HIERARCHY_COL].str.split('/', expand=True)

    # 2. Rename the new columns to Level 1, Level 2, etc.
    # This automatically names them based on how many levels were found
    num_levels = split_columns.shape[1]
    new_col_names = [f"Category Level {i+1}" for i in range(num_levels)]
    split_columns.columns = new_col_names

    # 3. Join the new columns back to the original data
    # We drop the original hierarchy column to keep it clean (remove .drop part if you want to keep it)
    df_clean = pd.concat([df, split_columns], axis=1)

    # 4. Save to Excel
    print(f"Saving to {OUTPUT_FILE}...")
    df_clean.to_excel(OUTPUT_FILE, index=False)
    print("Done! Here is a preview of the new columns:")
    print(df_clean[new_col_names].head())

if __name__ == "__main__":
    split_hierarchies

split_hierarchies()



Reading categories.xlsx...
Splitting hierarchies...
Saving to categories_cleaned.xlsx...
Done! Here is a preview of the new columns:
              Category Level 1  Category Level 2  \
0  Mineral building materials           Binders    
1        Insulation materials             Straw    
2                      Metals    Steel and iron    
3        Insulation materials         Hemp fibre   
4        Insulation materials      In‑situ foam    

                                Category Level 3 Category Level 4  
0                                         Cement             None  
1                                    Straw bales             None  
2   Cast and forged parts made of steel and iron             None  
3                                           None             None  
4                                        Mineral             None  


In [ ]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
import os

# --- CONFIGURATION ---
# Make sure these match your actual file names
PRODUCT_FILE = 'EPDs_Missing_Categories.xlsx' 
CATEGORY_FILE = 'categories_cleaned.xlsx'
OUTPUT_FILE = 'Mapped_EPD_Results.xlsx'

CONFIDENCE_THRESHOLD = 0.80

# --- SMART LOADER: Handles CSV or Excel ---
def load_file(filename):
    if not os.path.exists(filename):
        # Try appending .csv if the user forgot it, or check for " - Sheet1.csv" variants
        if os.path.exists(filename + " - Missing Categories.csv"):
            filename = filename + " - Missing Categories.csv"
        elif os.path.exists(filename + " - Sheet1.csv"):
            filename = filename + " - Sheet1.csv"
        else:
            raise FileNotFoundError(f"Could not find file: {filename}")
            
    if filename.endswith('.csv'):
        return pd.read_csv(filename)
    elif filename.endswith('.xlsx'):
        return pd.read_excel(filename)
    else:
        raise ValueError("File must be .csv or .xlsx")

def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'sku:\s*\S+', '', text)
    text = re.sub(r'[!@#$%^&*_{};:,.<>?|`~]', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

def main():
    print("--- STARTING MAPPER ---")

    # 1. LOAD FILES
    try:
        print("Loading files...")
        df_prods = load_file(PRODUCT_FILE)
        df_cats = load_file(CATEGORY_FILE)
    except Exception as e:
        print(f"Error: {e}")
        return

    # 2. DETECT PRODUCT COLUMN
    candidates = ['product name', 'name', 'name (en)', 'flow', 'ref']
    prod_col = None
    for col in df_prods.columns:
        for cand in candidates:
            if cand.lower() in col.lower():
                prod_col = col
                break
        if prod_col: break
    
    if not prod_col:
        print("Could not find product column. Using first column.")
        prod_col = df_prods.columns[0]
    
    print(f"   Using product column: {prod_col}")

    # 3. PREPARE CATEGORIES
    print("Preparing categories...")
    # Join all columns in the category file to make a path
    cat_paths = df_cats.apply(lambda x: ' > '.join(x.dropna().astype(str)), axis=1).tolist()
    unique_cats = list(set(cat_paths))

    # 4. ENCODE & MATCH
    print("Loading transformer model...")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    print("Encoding data...")
    cleaned_names = [clean_text(str(x)) for x in df_prods[prod_col]]
    prod_emb = model.encode(cleaned_names, convert_to_tensor=True)
    cat_emb = model.encode(unique_cats, convert_to_tensor=True)

    print("Matching...")
    cosine_scores = util.cos_sim(prod_emb, cat_emb)
    top_scores, top_indices = cosine_scores.max(dim=1)

    # 5. SAVE RESULTS
    results = []
    for i, name in enumerate(df_prods[prod_col]):
        score = top_scores[i].item()
        cat_idx = top_indices[i].item()
        
        results.append({
            "Original Product": name,
            "Matched Category": unique_cats[cat_idx],
            "Confidence": round(score, 4),
            "Status": "Review Needed" if score < CONFIDENCE_THRESHOLD else "Auto-Assigned"
        })

    df_res = pd.DataFrame(results)

    # 6. PRINT STATISTICS (NEW SECTION)
    print("\n" + "="*30)
    print("       MAPPING SUMMARY       ")
    print("="*30)
    
    total_count = len(df_res)
    status_counts = df_res['Status'].value_counts()
    
    auto_count = status_counts.get('Auto-Assigned', 0)
    review_count = status_counts.get('Review Needed', 0)
    
    print(f"Total Products Processed: {total_count}")
    print("-" * 30)
    print(f"✅ Auto-Assigned:  {auto_count}  ({(auto_count/total_count)*100:.1f}%)")
    print(f"⚠️  Review Needed:  {review_count}  ({(review_count/total_count)*100:.1f}%)")
    print("="*30 + "\n")

    # 7. SAVE RESULTS
    # Split categories back into levels
    split_cats = df_res['Matched Category'].str.split(' > ', expand=True)
    split_cats.columns = [f"Level {i+1}" for i in range(split_cats.shape[1])]
    
    final_df = pd.concat([df_res, split_cats], axis=1)
    final_df.to_excel(OUTPUT_FILE, index=False)
    print(f"--- Done! Results saved to {OUTPUT_FILE} ---")

if __name__ == "__main__":
    main()

c:\Users\user\.conda\envs\brightway\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- STARTING MAPPER ---
Loading files...
   Using product column: Unnamed: 0
Preparing categories...
Loading transformer model...
Encoding data...
Matching...

       MAPPING SUMMARY       
Total Products Processed: 4186
------------------------------
✅ Auto-Assigned:  2896  (69.2%)
⚠️  Review Needed:  1290  (30.8%)

--- Done! Results saved to Mapped_EPD_Results.xlsx ---


In [ ]:
import pandas as pd
import numpy as np
import re
import os
from sentence_transformers import SentenceTransformer, util

# --- CONFIGURATION ---
PRODUCT_FILE = 'EPDs_Missing_Categories.xlsx' 
CATEGORY_FILE = 'categories_cleaned.xlsx'
OUTPUT_FILE = 'Mapped_EPD_Results.xlsx'
STATS_FILE = 'Operator_Statistics.xlsx'
COUNTS_FILE = 'EPD_Counts_Per_Category_Per_Operator.xlsx'

CONFIDENCE_THRESHOLD = 0.80

def load_file(filename):
    """Smart loader for CSV or Excel."""
    if not os.path.exists(filename):
        # Handle common file name variations
        if os.path.exists(filename + " - Missing Categories.csv"):
            filename = filename + " - Missing Categories.csv"
        elif os.path.exists(filename + " - Sheet1.csv"):
            filename = filename + " - Sheet1.csv"
    
    print(f"Loading: {filename}")
    if filename.endswith('.csv'):
        return pd.read_csv(filename)
    elif filename.endswith('.xlsx'):
        return pd.read_excel(filename)
    else:
        raise ValueError("File must be .csv or .xlsx")

def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'sku:\s*\S+', '', text)
    text = re.sub(r'[!@#$%^&*_{};:,.<>?|`~]', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

def main():
    print("--- STARTING MAPPER (SentenceTransformer) ---")

    # 1. LOAD FILES
    try:
        df_prods = load_file(PRODUCT_FILE)
        df_cats = load_file(CATEGORY_FILE)
    except Exception as e:
        print(f"Error: {e}")
        return

    # 2. DETECT COLUMNS
    # Product Name
    candidates = ['product name', 'name', 'name (en)', 'flow', 'ref']
    prod_col = None
    for col in df_prods.columns:
        if any(cand.lower() in col.lower() for cand in candidates):
            prod_col = col
            break
    if not prod_col: prod_col = df_prods.columns[0]
    print(f"Using product column: {prod_col}")

    # Registration Authority
    auth_col = None
    for col in df_prods.columns:
        if "registration" in col.lower() and "authority" in col.lower():
            auth_col = col
            break
    if auth_col:
        print(f"Using Authority column: {auth_col}")

    # 3. PREPARE CATEGORIES
    # Create rich path for matching: "Level 1 > Level 2 > Level 3"
    cat_paths = df_cats.apply(lambda x: ' > '.join(x.dropna().astype(str)), axis=1).tolist()
    
    # Identify clean target name for the final file
    if 'Category (en)' in df_cats.columns:
        target_labels = df_cats['Category (en)'].tolist()
    else:
        target_labels = cat_paths

    # 4. ENCODE & MATCH
    print("Loading model...")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    print("Encoding data (this may take a moment)...")
    cleaned_names = [clean_text(str(x)) for x in df_prods[prod_col]]
    
    # Encode
    prod_emb = model.encode(cleaned_names, convert_to_tensor=True)
    cat_emb = model.encode(cat_paths, convert_to_tensor=True)

    print("Matching...")
    cosine_scores = util.cos_sim(prod_emb, cat_emb)
    top_scores, top_indices = cosine_scores.max(dim=1)

    # 5. BUILD RESULTS
    results = []
    for i, name in enumerate(df_prods[prod_col]):
        score = top_scores[i].item()
        cat_idx = top_indices[i].item()
        
        row_data = {
            "Original Product": name,
            "Matched Category": target_labels[cat_idx], # Clean single column
            "Matched Path": cat_paths[cat_idx],         # Full path (optional, helpful for review)
            "Confidence": round(score, 4),
            "Status": "Review Needed" if score < CONFIDENCE_THRESHOLD else "Auto-Assigned"
        }
        if auth_col:
            row_data['Registration Authority'] = df_prods.iloc[i][auth_col]
            
        results.append(row_data)

    df_res = pd.DataFrame(results)

    # 6. STATISTICS & COUNTS
    if auth_col:
        print("\n--- Generating Statistics ---")
        
        # A. Unique Categories per Operator
        auth_unique = df_res.groupby('Registration Authority')['Matched Category'].nunique().reset_index()
        auth_unique.columns = ['Registration Authority', 'Unique Categories']
        auth_unique.to_excel(STATS_FILE, index=False)
        
        # B. Detailed Counts (Number of EPDs per Category per Operator)
        # Group by Authority AND Category, then count rows
        cat_counts = df_res.groupby(['Registration Authority', 'Matched Category']).size().reset_index(name='Number of EPDs')
        cat_counts = cat_counts.sort_values(['Registration Authority', 'Number of EPDs'], ascending=[True, False])
        
        cat_counts.to_excel(COUNTS_FILE, index=False)
        print(f"Detailed counts saved to: {COUNTS_FILE}")

    # 7. SAVE MAIN RESULTS
    df_res.to_excel(OUTPUT_FILE, index=False)
    print(f"Main results saved to: {OUTPUT_FILE}")
    print("\nDone.")

if __name__ == "__main__":
    main()

--- STARTING MAPPER (SentenceTransformer) ---
Loading: EPDs_Missing_Categories.xlsx
Loading: categories_cleaned.xlsx
Using product column: Unnamed: 0
Using Authority column: Registration authority
Loading model...
Encoding data (this may take a moment)...
Matching...

--- Generating Statistics ---
Detailed counts saved to: EPD_Counts_Per_Category_Per_Operator.xlsx
Main results saved to: Mapped_EPD_Results.xlsx

Done.


In [22]:
# number of EPDs assigned product category grouped by program operator
import pandas as pd
import os

FILE = 'Mapped_EPD_Results.xlsx'
if not os.path.exists(FILE):
    raise FileNotFoundError(f'{FILE} not found. Run the mapper to produce mapping results.')
df1 = pd.read_excel(FILE)

# Detect registration authority column name (case-insensitive)
reg_col = None
for c in df1.columns:
    if c.lower().strip() == 'registration authority':
        reg_col = c
        break
if reg_col is None:
    for c in df1.columns:
        if 'registration' in c.lower() and 'authority' in c.lower():
            reg_col = c
            break
if reg_col is None:
    # fallback to a canonical name and create it if missing
    reg_col = 'Registration Authority'
    df1[reg_col] = pd.NA

# Normalize a clean column for grouping
df1['Registration_Authority_Clean'] = df1[reg_col].astype(str).str.strip()
# Treat empty strings and common placeholders as missing
df1['Registration_Authority_Clean'] = df1['Registration_Authority_Clean'].replace({'': pd.NA, 'nan': pd.NA})
df1['Registration_Authority_Clean'] = df1['Registration_Authority_Clean'].fillna('Unknown')

# Counts by operator and status
assigned_counts = df1[df1['Status'] == 'Auto-Assigned'].groupby('Registration_Authority_Clean').size().reset_index(name='Number of Auto-Assigned EPDs')
review_counts = df1[df1['Status'] == 'Review Needed'].groupby('Registration_Authority_Clean').size().reset_index(name='Number of Review Needed EPDs')

print('Auto-Assigned by Registration Authority:')
print(assigned_counts.head(50).to_string(index=False))

print('\nReview Needed by Registration Authority:')
print(review_counts.head(50).to_string(index=False))

# Counts where registration authority is missing/unknown
missing_mask = df1[reg_col].isna() | df1[reg_col].astype(str).str.strip().eq('') | df1[reg_col].astype(str).str.lower().eq('unknown')
missing_total = missing_mask.sum()
print(f'\nEPDs without a program operator (missing/unknown): {missing_total}')
missing_by_status = df1[missing_mask].groupby('Status').size().reindex(['Auto-Assigned','Review Needed']).fillna(0).astype(int).reset_index(name='Number of EPDs')
print(missing_by_status.to_string(index=False))

# Save a small Excel summary for missing operators
missing_by_status.to_excel('mapping_missing_operator_counts.xlsx', index=False)
print('Saved mapping_missing_operator_counts.xlsx')

Auto-Assigned by Registration Authority:
Registration_Authority_Clean  Number of Auto-Assigned EPDs
  AENOR Internacional S.A.U.                             1
           DAPHabitat System                            39
                 EPD Denmark                           711
                 EPD Ireland                            96
                  EPD Norway                            31
               EPDFinlandRTS                            35
                    EPDItaly                          1045
                 ICMQ S.p.A.                             7
              ITB-EPD Poland                           157
    Programa DAPconstrucción                           104
             Stichting MRPI®                           322
            Stichting MRPIÂ®                            13
                     Unknown                           335

Review Needed by Registration Authority:
Registration_Authority_Clean  Number of Review Needed EPDs
           DAPHabitat System    